# Github Survey Analysis Python Code

In [2]:
from pandas import *
import pandas as pd
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import datetime as dt
import read3 as rd
from read3 import *
today = dt.date.today().strftime("%Y%m%d")
import math
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 6)
import statsmodels.api as sm

from scipy import stats
import os 
import read3 as rd

import plotly.offline as offline
from plotly.offline import download_plotlyjs, iplot
import plotly.graph_objs as go
from plotly.graph_objs import  *
import plotly.tools as tls
#from plotly.offline.offline import *

offline.init_notebook_mode(connected=True)

In [3]:
df = pd.read_csv('C:/Users/jiaqil/Desktop/Things/github/nps-exercise.csv')

In [4]:
df.columns.values


array(['id', 'github_age_in_days', 'github_in_org', 'github_in_pade_org',
       'github_personal_paid', 'status', 'userAgent', 'answer-3',
       'answer-9', 'answer-10', 'answer-11', 'answer-12', 'answer-14',
       'answer-15', 'answer-45', 'answer-58', 'answer-59', 'answer-60',
       'answer-62', 'answer-63'], dtype=object)

In [5]:
[min(df['github_age_in_days']),max(df['github_age_in_days']),df['github_age_in_days'].mean()]
len(df['github_age_in_days'].dropna())

1600

In [6]:
df= df.rename(columns={'answer-3':'Q1_Role'})
df= df.rename(columns={'answer-10':'Q5_Score'})
df= df.rename(columns={'answer-58':'Q2_Impression'})
df= df.rename(columns={'answer-14':'Q3_Improve'})

# NPS Analysis

Recommendation Rating Distributions

In [7]:
nps_age=df[['github_age_in_days','Q5_Score']]
nps_age = nps_age.sort(['github_age_in_days'], ascending=1)
#nps_age=nps_age.reset_index()
nps_age = nps_age.dropna(subset=(['Q5_Score']))
dist_score = pd.DataFrame(nps_age.groupby(['Q5_Score']).count()).reset_index()
x = list(dist_score['Q5_Score'])
y=list(dist_score['github_age_in_days'])
data = [go.Bar(
    x=dist_score['Q5_Score'],
    y=dist_score['github_age_in_days'],
    marker=dict(color=['#D61800','#D61800','#D61800','#D61800','#D61800','#D61800',
                       '#D61800','#FAAF08','#FAAF08','#31A9B8','#31A9B8']),
)]
layout = go.Layout(
    title="NPS Rating Responses Distribution",
    xaxis=dict(title='NPS Rating',tickmode = 'linear', categoryorder='array', side='bottom',type="category"),
    width=700,
    height=400,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='Count'
        ),
    font=dict(size=14),
    annotations=[
        dict(x=xi,y=yi,
             text=str(yi),
             xanchor='center',
             yanchor='bottom',
             showarrow=False,
        ) for xi, yi in zip(x,y)]
)


fig = go.Figure(data=data,layout=layout)

iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=850,image='png')
nps_hist = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=850, image_height=500)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



# NPS Calucation

In [382]:
total_sample = len(nps_age['Q5_Score']) # total: 1593, 7 out of 1600 are null

promoters = len([i for i in list(nps_age['Q5_Score']) if i >= 9]) #1060
passives =len([i for i in list(nps_age['Q5_Score']) if i < 9  and i > 6]) #429
detractors= len([i for i in list(nps_age['Q5_Score']) if i <= 6]) #104

prom_percent=float(promoters/total_sample) #0.665
pass_percent = float(passives/total_sample) #0.269
detr_percent = float(detractors/total_sample) #0.066

NPS = prom_percent -detr_percent #0.6001
NPS 

0.6001255492780916

# Margin of Error

In [430]:
#sample proprtion
Var_NPS = ((1-NPS)**2)*prom_percent + ((0-NPS)**2)*pass_percent + ((-1-NPS)**2)*detr_percent
#0.3705
MOE = ((Var_NPS)**(1/2))/((total_sample)**(1/2)) #1.525%

#95% confidence interval
[NPS-1.96*MOE,NPS+1.96*MOE]

print(str(MOE*100) +'%')

1.5251524238133176%


# Account Age Analysis

In [384]:
import seaborn as sns

ax1 = sns.boxplot(x='Q5_Score', y='github_age_in_days',data=nps_age)
#sns.plt.show(ax1)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [8]:
age1=list(df['github_age_in_days'])
count=[376,358,347,280,137,69,33]
#xaxis = [182.5,547.5,912.5,1277.5,1642.5,2007.5,2372.5]
xaxis=['1 Year','2 Year','3 Year','4 Year','5 Year','6 Year','7 Year']

data = [go.Bar(
    x=xaxis,
    y= count,
    marker=dict(
        color='#31A9B8',
    ),
    #opacity=0.75,
)]

layout = go.Layout(
    width= 600,
    height = 400,
    title='Github Accoun Age Distribution ',
    #xaxis=dict(
       # title='Github Account Age (Year)'
    #),
    yaxis=dict(
        title='Count'
    ),
   annotations=[
        dict(x=xi,y=yi,
             font=dict(
                size=10
            ),
             text=str(yi),
             xanchor='center',
             yanchor='top',
             showarrow=False,
        ) for xi, yi in zip(xaxis, count)],
    bargap=0.1,
    bargroupgap=0.1,
    font = dict(size=14),
    titlefont = dict(size=18),
    margin=dict(
            t=50,
            #autoexpand=True
        ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='styled histogram',show_link=False)
#offline.plot(fig,image_height=400,image_width=600,image='png')
age_hist = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=600, image_height=400)

In [423]:
age1=list(df['github_age_in_days'])
count=[376,358,347,280,137,69,33]
xaxis = [182.5,547.5,912.5,1277.5,1642.5,2007.5,2372.5]
data = [go.Histogram(
    x=age1,
    histfunc='count',
    #name='',
    autobinx=False,
    #text=['22.2%','19.1%','16.5%','22.6%','19.6%'],
    xbins=dict(
        start=0,
        end=2500,
        size=365
    ),
    marker=dict(
        color='#31A9B8',
    ),
    #opacity=0.75,
)]

layout = go.Layout(
    width= 600,
    height = 400,
    title='Github Accoun Age Distribution ',
    xaxis=dict(
        title='Github Age (Days)'
    ),
    yaxis=dict(
        title='Count'
    ),
     annotations=[
        dict(x=xi,y=yi,
             font=dict(
                size=10
            ),
             text=str(yi),
             xanchor='center',
             yanchor='top',
             showarrow=False,
        ) for xi, yi in zip(xaxis, count)],
    bargap=0.1,
    bargroupgap=0.1,
    font = dict(size=14),
    titlefont = dict(size=18),
    margin=dict(
            t=50,
            #autoexpand=True
        ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='styled histogram',show_link=False)
offline.plot(fig,image_height=400,image_width=600,image='png')

'file://C:\\Users\\jiaqil\\Documents\\Python Scripts\\temp-plot.html'

# categorize customer ages into half year & a year level

In [9]:
def agebucket1(row):
    if row <= 180:
        return "H1"
    if row < 180*2:
        return "H2"
    if row <= 180*3:
        return "H3"
    if row <= 180*4:
        return "H4"
    if row <= 180*5:
        return "H5"
    if row <= 180*6:
        return "H6"
    if row <= 180*7:
        return "H7"
    if row <= 180*8:
        return "H8"
    if row <= 180*9:
        return "H9"
    if row <= 180*10:
        return "H10"
    if row <= 180*11:
        return "H11"
    if row <= 180*12:
        return "H12"
    if row <= 180*13:
        return "H13"
    if row <= 180*14:
        return "H14"

def agebucket2(row):
    if row <=365:
        return "Y1"
    if row <= 365*2:
        return "Y2"
    if row <= 365*3:
        return "Y3"
    if row <365*4:
        return "Y4"
    if row <=365*5:
        return "Y5"
    if row <=365*6:
        return "Y6"
    if row <=365*7:
        return "Y7"
    
nps_age['Age_Year1'] = nps_age['github_age_in_days'].apply(lambda x : agebucket1(x))
nps_age['Age_Year2'] = nps_age['github_age_in_days'].apply(lambda x : agebucket2(x))
nps_age

,github_age_in_days,Q5_Score,Age_Year1,Age_Year2
739,1.51,8.0,H1,Y1
1591,1.53,7.0,H1,Y1
486,1.78,5.0,H1,Y1
...,...,...,...,...
1503,2426.59,10.0,H14,Y7
10,2427.56,8.0,H14,Y7
941,2446.51,10.0,H14,Y7


# NPS for Accout Age Groups (year & half year)

In [10]:
levels1 = nps_age['Age_Year1'].unique()
levels2 = nps_age['Age_Year2'].unique()

def nps_agelevel(df,levels,column):
    lst=[]
    for level in list(levels):
        df1 = df[df[column]==level]
        total_sample = len(df1['Q5_Score'])
        promoters = len([i for i in list(df1['Q5_Score']) if i >= 9]) 
        passives =len([i for i in list(df1['Q5_Score']) if i < 9  and i > 6])
        detractors= len([i for i in list(df1['Q5_Score']) if i <= 6])
        prom_percent=float(promoters/total_sample)
        detr_percent = float(detractors/total_sample)
        pass_percent = float(passives/total_sample)
        NPS = prom_percent -detr_percent
        lst.append([level,NPS,total_sample,promoters,prom_percent,passives,pass_percent,detractors,detr_percent])
    return pd.DataFrame(lst,columns=('Age_Year','NPS','Total','Promoters','Promoter%','Passives','Passive%',
                                     'Detractors','Detractor%'))

agelevel1= nps_agelevel(nps_age,levels1,'Age_Year1')
agelevel2= nps_agelevel(nps_age,levels2,'Age_Year2')
agelevel1
agelevel2

,Age_Year,NPS,Total,Promoters,Promoter%,Passives,Passive%,Detractors,Detractor%
0,Y1,0.459893,374,210,0.561497,126,0.336898,38,0.101604
1,Y2,0.566197,355,229,0.645070,98,0.276056,28,0.078873
2,Y3,0.670520,346,250,0.722543,78,0.225434,18,0.052023
...,...,...,...,...,...,...,...,...,...
4,Y5,0.627737,137,93,0.678832,37,0.270073,7,0.051095
5,Y6,0.724638,69,52,0.753623,15,0.217391,2,0.028986
6,Y7,0.606061,33,22,0.666667,9,0.272727,2,0.060606


In [11]:
# Stack Bar Plot of propotions of promoters%, passive% and detractor% for diff Age
layout = go.Layout(
    title="NPS with Account Age (Year)",
    xaxis=dict(title='NPS Rating',tickmode = 'linear', categoryorder='array', side='bottom',type="category"),
    width=700,
    height=500,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='NPS Proportion'
        ),
    font=dict(size=14))

def agebardata(df,column):
    data=[]
    data1 = go.Bar(
        x=df[column],
        y=df['Promoter%'],
        marker=dict(color='#31A9B8'),
        name = 'Promoter%')
    data2 = go.Bar(
        x=df[column],
        y=df['Passive%'],
        marker=dict(color='#FAAF08'),
        name ='Passive%')
    data3 = go.Bar(
        x=df[column],
        y=df['Detractor%'],
        marker=dict(color='#D61800'),
        name ='Detractor%')
    data.append(data1)
    data.append(data2)
    data.append(data3)
    return data

data4 = go.Scatter(
                  x=agelevel2['Age_Year'], 
                  y=agelevel2['NPS'], 
                  mode='lines+markers',
                  marker=go.Marker(color='#7c7c7c'),
                  name='NPS Score'
                  )
data=agebardata(agelevel2,'Age_Year')
data.append(data4)
fig = go.Figure(data=data,layout=layout)
fig['layout'].update(barmode='stack')
iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=750,image='png')
agenps = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=750, image_height=500)

In [12]:

line = [0.0165*i+0.5042 for i in range(0,15)]


data1 = go.Scatter(
    x=agelevel1['Age_Year'],
    y=agelevel1['NPS'],
    mode='markers',
    marker=dict(color='#31A9B8',size=10),
    name = 'NPS by Age: 180 days'
)

data2 = go.Scatter(
                  x=agelevel1['Age_Year'], 
                  y=line, 
                  mode='lines',
                  marker=go.Marker(color='#D61800'),
                  name='Linear Regression Fit'
                  )
#data =[data1]
data = [data1,data2]
layout = go.Layout(
    title="NPS by Account Age: Group by Half Year",
    width=750,
    height=400,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='NPS'
        ),
    xaxis=dict(
            title='Age (Half Year)'
        ),
    font=dict(size=14),
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,show_link=False)
#offline.plot(fig,image_height=400,image_width=750,image='png')

In [13]:
line = [i*0.0255+0.5202 for i in range(0,8)]

data1 = go.Scatter(
    x=agelevel2['Age_Year'],
    y=agelevel2['NPS'],
    mode='markers',
    marker=dict(color='#31A9B8',size=12),
    name = 'NPS by Age: a Year'
)


data2 = go.Scatter(
                  x=agelevel2['Age_Year'], 
                  y=line, 
                  mode='lines',
                  marker=go.Marker(color='#D61800'),
                  name='Linear Regression Fit'
                  )

data = [data1,data2]
#data =[data1]
layout = go.Layout(
    title="NPS by Account Age: Grouped by Year",
    width=750,
    height=450,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='NPS'
        ),
    xaxis=dict(
            title='Age (Year)'
        ),
    font=dict(size=14),
)
fig = go.Figure(data=data,layout=layout)
iplot(fig,show_link=False)
#offline.plot(fig,image_height=450,image_width=750,image='png')
linear = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=750, image_height=450)

# Linear Regression Fitting on the Age vs. NPS data

In [14]:
### Half Year
import statsmodels.api as sm
import statsmodels.formula.api as smf
agelevel1['Age_Year_num'] = agelevel1['Age_Year'].apply(lambda x: x[1:])
agelevel1['Age_Year_num']=pd.to_numeric(agelevel1['Age_Year_num'],errors='coerce')
reg1 = smf.ols('NPS ~ Age_Year_num', data=agelevel1).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                    NPS   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     11.80
Date:                Mon, 17 Apr 2017   Prob (F-statistic):            0.00494
Time:                        11:08:21   Log-Likelihood:                 17.939
No. Observations:                  14   AIC:                            -31.88
Df Residuals:                      12   BIC:                            -30.60
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        0.5042      0.041     12.308   

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\scipy\stats\stats.py:1535: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=14



In [15]:
# Group by Year
import statsmodels.api as sm
import statsmodels.formula.api as smf
agelevel2['Age_Year_num'] = agelevel2['Age_Year'].apply(lambda x: x[1:])
agelevel2['Age_Year_num']=pd.to_numeric(agelevel2['Age_Year_num'],errors='coerce')
reg1 = smf.ols('NPS ~ Age_Year_num', data=agelevel2).fit()
print(reg1.summary())

                            OLS Regression Results                            
Dep. Variable:                    NPS   R-squared:                       0.374
Model:                            OLS   Adj. R-squared:                  0.249
Method:                 Least Squares   F-statistic:                     2.988
Date:                Mon, 17 Apr 2017   Prob (F-statistic):              0.144
Time:                        11:08:22   Log-Likelihood:                 9.1105
No. Observations:                   7   AIC:                            -14.22
Df Residuals:                       5   BIC:                            -14.33
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        0.5202      0.066      7.900   

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning:

omni_normtest is not valid with less than 8 observations; 7 samples were given.



# Anova Test on Raw Response Ratings

In [16]:
#Since Age Level is categorical variable with 10 or 5 levels
age_mean1 = pd.DataFrame(nps_age.groupby(['Age_Year1'])['Q5_Score'].mean()).reset_index()
age_mean2 = pd.DataFrame(nps_age.groupby(['Age_Year2'])['Q5_Score'].mean()).reset_index()

In [17]:
#ANOVA
model1 = smf.ols(formula = 'Q5_Score ~C(Age_Year1)',data=nps_age).fit()
print(model1.summary())

model2 = smf.ols(formula = 'Q5_Score ~C(Age_Year2)',data=nps_age).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:               Q5_Score   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.741
Date:                Mon, 17 Apr 2017   Prob (F-statistic):           6.36e-06
Time:                        11:08:26   Log-Likelihood:                -2954.2
No. Observations:                1593   AIC:                             5936.
Df Residuals:                    1579   BIC:                             6012.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
Intercept               8.3547    

In [18]:
#POST HOC Test: TUKEY HSD Test

from scipy import stats
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,MultiComparison)
mc1 = MultiComparison(nps_age['Q5_Score'],nps_age['Age_Year2'])
res1=mc1.tukeyhsd()
print(res1.summary())

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  Y1     Y2    0.3009  -0.0393 0.641  False 
  Y1     Y3    0.5244   0.182  0.8668  True 
  Y1     Y4    0.6202   0.257  0.9833  True 
  Y1     Y5    0.4621   0.0036 0.9205  True 
  Y1     Y6    0.7158   0.1144 1.3173  True 
  Y1     Y7    0.5143  -0.3193 1.3479 False 
  Y2     Y3    0.2236  -0.1232 0.5703 False 
  Y2     Y4    0.3193   -0.048 0.6866 False 
  Y2     Y5    0.1612  -0.3005 0.6229 False 
  Y2     Y6    0.415    -0.189 1.0189 False 
  Y2     Y7    0.2134   -0.622 1.0488 False 
  Y3     Y4    0.0957  -0.2736 0.4651 False 
  Y3     Y5   -0.0624  -0.5257 0.401  False 
  Y3     Y6    0.1914  -0.4138 0.7966 False 
  Y3     Y7   -0.0102  -0.8465 0.8262 False 
  Y4     Y5   -0.1581   -0.637 0.3208 False 
  Y4     Y6    0.0957  -0.5215 0.7129 False 
  Y4     Y7   -0.1059  -0.9509 0.7391 False 
  Y5     Y6    0.2538  -0.4239 0.9314 False 
  Y5

In [19]:
year=pd.DataFrame(nps_age.groupby(['Age_Year2'])['Q5_Score'].mean()).reset_index()
data = [go.Bar(
    y=year['Q5_Score'],
    x=year['Age_Year2'],
    marker=dict(color=['#D61800','#31A9B8','#31A9B8','#31A9B8','#31A9B8','#31A9B8','#31A9B8']),
)]
layout = go.Layout(
    title="Ratings (Mean) vs. Account Age (Year)",
    xaxis=dict(title='Age in Year',tickmode = 'linear', categoryorder='array', side='bottom',type="category"),
    width=500,
    height=350,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='Rating Score(mean)'
        ),
    bargap=0.5,
    font=dict(size=12))


fig = go.Figure(data=data,layout=layout)

iplot(fig,show_link=False)
offline.plot(fig,image_height=400,image_width=650,image='png')

'file://C:\\Users\\jiaqil\\Documents\\Python Scripts\\temp-plot.html'

# Customer Role Data Investigation

In [20]:
role2 = pd.DataFrame(df.groupby('Q1_Role').size().reset_index())
labels=list(role2['Q1_Role'])
values=list(role2[0])
text=list(role2['Q1_Role'])
color =['#258039','#31A9B8','#F5BE41','#FA6E59','#4D648D','#C9A66B']
data=[go.Pie(showlegend=False,values=values,text =text, textposition='outside',marker=dict(colors=color))]

layout = go.Layout(
    width= 700,
    height = 400,
    title='Github Roles Distribution',
    font = dict(size=16),
    titlefont = dict(size=20),
    margin=dict(
            t=50,
            #autoexpand=True
        ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=800,image='png')
role_pie = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=800, image_height=500)

In [21]:
role = df[['github_age_in_days','Q1_Role','Q5_Score']]
roleslst = role['Q1_Role'].dropna().unique()

def nps_rolelevel(df,levels,column):
    lst=[]
    for level in list(levels):
        df1 = df[df[column]==level]
        total_sample = len(df1['Q5_Score'])
        promoters = len([i for i in list(df1['Q5_Score']) if i >= 9]) 
        passives =len([i for i in list(df1['Q5_Score']) if i < 9  and i > 6])
        detractors= len([i for i in list(df1['Q5_Score']) if i <= 6])
        prom_percent=float(promoters/total_sample)
        detr_percent = float(detractors/total_sample)
        pass_percent = float(passives/total_sample)
        NPS = prom_percent -detr_percent
        lst.append([level,NPS,total_sample,promoters,prom_percent,passives,pass_percent,detractors,detr_percent])
    return pd.DataFrame(lst,columns=('Role','NPS','Total','Promoters','Promoter%','Passives','Passive%',
                                     'Detractors','Detractor%'))

nps_role= nps_rolelevel(role, roleslst,'Q1_Role')
nps_role = nps_role.sort(['NPS'], ascending=0)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [22]:
layout = go.Layout(
    title="NPS with Customer Role",
    xaxis=dict(tickmode = 'linear', categoryorder='array', side='bottom',type="category"),
    width=700,
    height=500,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='NPS Proportion'
        ),
    font=dict(size=12),
    margin=dict(
            b=150,
            l=100,
            r=50,
            t=50,
            #autoexpand=True
        ),)

data=agebardata(nps_role,'Role')
        
data4 = go.Scatter(
                  x=nps_role['Role'], 
                  y=nps_role['NPS'], 
                  mode='lines+markers',
                  marker=go.Marker(color='#7c7c7c'),
                  name='NPS Score'
                  )
data.append(data4)
fig = go.Figure(data=data,layout=layout)
fig['layout'].update(barmode='stack')
iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=750,image='png')
rolenps = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=750, image_height=500)

In [23]:
role_age = pd.DataFrame(role.groupby(['Q1_Role']).mean()).reset_index()
role_age=role_age.sort(['github_age_in_days'], ascending=0)
role_age_nps = pd.merge(nps_role,role_age,left_on='Role',right_on='Q1_Role')
role_age_nps=role_age_nps.sort(['github_age_in_days'], ascending=0)

data0 = go.Bar(
    x=role_age_nps['Q1_Role'],
    y=role_age_nps['github_age_in_days'],
    marker=dict(
        color='#F5BE41',
    ),
    name='Mean Account Age of Diff Roles',
)
data1 = go.Scatter(
    x=role_age_nps['Q1_Role'],
    y=role_age_nps['NPS'],
    mode='lines+markers',
    line=dict(
        color='#31A9B8'),
    name='NPS Score of Diff Roles',
    yaxis='y2'
)

layout = go.Layout(
        autosize =False,
        width=850,
        height=450,
        title='Mean Account Age & NPS Score of Diff Roles',
        legend=Legend(
            x=1.15,
            y=0.5,
            bgcolor='transparent',
            bordercolor='transparent',
            borderwidth=0,
            font=Font(
                color='',
                family='',
                size=12
            ),
            traceorder='normal',
            xanchor='left',
            yanchor='auto'
        ),
        
        font = dict(size=14),
        titlefont = dict(size=18),
        margin=dict(
            b=150,
            l=100,
            r=50,
            t=50,
            #autoexpand=True
        ),
        yaxis=dict(
            title='#Days using Github'
        ),
        yaxis2=dict(
            title='NPS Score',
            titlefont=dict(
                color='rgb(148, 103, 189)'
            ),
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            #autorange='reversed',
            overlaying='y',
            side='right'

        )
    )
data=[data1,data0]
#py.iplot(fig, filename='stacked-subplots')
fig = go.Figure(data=data,layout=layout)


iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=850,image='png')
meanagenps = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=850, image_height=500)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



# What you do think first of Github?

In [24]:
impress= df[['Q5_Score','Q2_Impression','github_age_in_days']]
impress1 = pd.DataFrame(df.groupby('Q2_Impression').size().reset_index())
labels=list(impress1['Q2_Impression'])
values=list(impress1[0])
text=list(impress1['Q2_Impression'])
color =['#258039','#F5BE41','#FA6E59','#4D648D','#C9A66B','#D61800']
data=[go.Pie(showlegend=False,values=values,text =text, textposition='outside',marker=dict(colors=color))]

layout = go.Layout(
    width= 750,
    height = 450,
    title='Fisrt Thought of Github',
    font = dict(size=14),
    titlefont = dict(size=18),
    margin=dict(
            t=100,
            #autoexpand=True
        ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=850,image='png')
first = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=850, image_height=500)

# Improvement Analysis

In [25]:
df= df.rename(columns={'answer-11':'Q6_Change'})
df= df.rename(columns={'answer-15':'Q11_Research'})
df= df.rename(columns={'answer-45':'Q3_Improve_SelText'})
df= df.rename(columns={'answer-59':'Q4_Improve_Text'})

In [26]:
improve = df[['github_age_in_days','Q1_Role','Q5_Score', 'Q6_Change','Q3_Improve','Q11_Research', 
              'Q3_Improve_SelText', 'Q2_Impression','Q4_Improve_Text']]

In [27]:
improve1 = pd.DataFrame(df.groupby('Q3_Improve').size().reset_index())
improve1 =improve1.sort([0], ascending=0)
labels=list(improve1 ['Q3_Improve'])
values=list(improve1 [0])
text=list(improve1['Q3_Improve'])
color =['#258039','#F5BE41','#FA6E59','#31A9B8','#4D648D','#baa896','#8593ae']
data=[go.Pie(showlegend=False,values=values,text =text, textposition='outside',marker=dict(colors=color))]

layout = go.Layout(
    width= 750,
    height = 450,
    title='What to Improve?',
    font = dict(size=14),
    titlefont = dict(size=18),
    margin=dict(
            t=100,
            #autoexpand=True
        ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=750,image='png')
imp_pie = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=850, image_height=500)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [28]:
implst = improve['Q3_Improve'].dropna().unique()
def nps_rolelevel(df,levels,column):
    lst=[]
    for level in list(levels):
        df1 = df[df[column]==level]
        total_sample = len(df1['Q5_Score'])
        promoters = len([i for i in list(df1['Q5_Score']) if i >= 9]) 
        passives =len([i for i in list(df1['Q5_Score']) if i < 9  and i > 6])
        detractors= len([i for i in list(df1['Q5_Score']) if i <= 6])
        prom_percent=float(promoters/total_sample)
        detr_percent = float(detractors/total_sample)
        pass_percent = float(passives/total_sample)
        NPS = prom_percent -detr_percent
        lst.append([level,NPS,total_sample,promoters,prom_percent,passives,pass_percent,detractors,detr_percent])
    return pd.DataFrame(lst,columns=('Improvement','NPS','Total','Promoters','Promoter%','Passives','Passive%',
                                     'Detractors','Detractor%'))

nps_improve= nps_rolelevel(improve, implst,'Q3_Improve')
nps_improve= nps_improve.sort(['NPS'], ascending=0)

nps_improve

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:19: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



,Improvement,NPS,Total,Promoters,Promoter%,Passives,Passive%,Detractors,Detractor%
0,Project management tools,0.697861,374,269,0.719251,97,0.259358,8,0.021390
3,Code review,0.640625,256,174,0.679688,72,0.281250,10,0.039062
2,GitHub desktop apps,0.609302,215,142,0.660465,62,0.288372,11,0.051163
...,...,...,...,...,...,...,...,...,...
5,Other (please specify),0.553903,269,177,0.657993,64,0.237918,28,0.104089
1,Help documentation,0.504132,121,75,0.619835,32,0.264463,14,0.115702
4,Pricing,0.502092,239,146,0.610879,67,0.280335,26,0.108787


In [29]:
layout = go.Layout(
    title="NPS with Improvement Aspects",
    xaxis=dict(tickmode = 'linear', categoryorder='array', side='bottom',type="category"),
    width=700,
    height=500,
    autosize=False,
    #yaxis=dict ( tickmode = 'linear',autorange="reversed",type="category")
    yaxis=dict(
            title='NPS Proportion'
        ),
    font=dict(size=12),
    margin=dict(
            b=150,
            l=100,
            r=50,
            t=50,
            #autoexpand=True
        ),)

data=agebardata(nps_improve,'Improvement')
        
data4 = go.Scatter(
                  x=nps_improve['Improvement'], 
                  y=nps_improve['NPS'], 
                  mode='lines+markers',
                  marker=go.Marker(color='#7c7c7c'),
                  name='NPS Score'
                  )
data.append(data4)
fig = go.Figure(data=data,layout=layout)
fig['layout'].update(barmode='stack')
iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=750,image='png')
impnps = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=750, image_height=500)

In [30]:
imp_age = pd.DataFrame(improve.groupby(['Q3_Improve'])['github_age_in_days'].mean()).reset_index()
imp_age_nps = pd.merge(nps_improve,imp_age,left_on='Improvement',right_on='Q3_Improve')
imp_age_nps=imp_age_nps.sort(['github_age_in_days'], ascending=0)

data0 = go.Bar(
    x=imp_age_nps['Improvement'],
    y=imp_age_nps['github_age_in_days'],
    marker=dict(
        color='#F5BE41',
    ),
    name='Mean Account Age of Improvements',
)
data1 = go.Scatter(
    x=imp_age_nps['Improvement'],
    y=imp_age_nps['NPS'],
    mode='lines+markers',
    line=dict(
        color='#31A9B8'),
    name='NPS Score of Improvements',
    yaxis='y2'
)

layout = go.Layout(
        autosize =False,
        width=850,
        height=450,
        title='Mean Account Age & NPS Score of Improvement Aspects',
        legend=Legend(
            x=1.15,
            y=0.5,
            bgcolor='transparent',
            bordercolor='transparent',
            borderwidth=0,
            font=Font(
                color='',
                family='',
                size=12
            ),
            traceorder='normal',
            xanchor='left',
            yanchor='auto'
        ),
        
        font = dict(size=14),
        titlefont = dict(size=18),
        margin=dict(
            b=150,
            l=100,
            r=50,
            t=50,
            #autoexpand=True
        ),
        yaxis=dict(
            title='#Days using Github'
        ),
        yaxis2=dict(
            title='NPS Score',
            titlefont=dict(
                color='rgb(148, 103, 189)'
            ),
            tickfont=dict(
                color='rgb(148, 103, 189)'
            ),
            #autorange='reversed',
            overlaying='y',
            side='right'

        )
    )
data=[data1,data0]
#py.iplot(fig, filename='stacked-subplots')
fig = go.Figure(data=data,layout=layout)


iplot(fig,show_link=False)
#offline.plot(fig,image_height=500,image_width=850,image='png')
impmeannps = offline.plot(fig, include_plotlyjs=False, output_type='div',show_link=False,image_width=750, image_height=500)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [31]:
imp_age_nps

,Improvement,NPS,Total,Promoters,Promoter%,Passives,Passive%,Detractors,Detractor%,Q3_Improve,github_age_in_days
6,Pricing,0.502092,239,146,0.610879,67,0.280335,26,0.108787,Pricing,950.855272
1,Code review,0.640625,256,174,0.679688,72,0.281250,10,0.039062,Code review,942.305391
0,Project management tools,0.697861,374,269,0.719251,97,0.259358,8,0.021390,Project management tools,925.711765
...,...,...,...,...,...,...,...,...,...,...,...
3,User administration tools,0.607477,107,71,0.663551,30,0.280374,6,0.056075,User administration tools,859.493271
2,GitHub desktop apps,0.609302,215,142,0.660465,62,0.288372,11,0.051163,GitHub desktop apps,668.163302
5,Help documentation,0.504132,121,75,0.619835,32,0.264463,14,0.115702,Help documentation,567.260992


# Segmentation Analysis

In [32]:
pd.set_option("display.max_rows", 8)
improve['Age_Year2'] = improve['github_age_in_days'].apply(lambda x : agebucket2(x))
#improve['Age_Improve']=improve['Age_Year2']+"_"+improve['Q1_Role']+"_"+improve['Q3_Improve']
improve['Age_Improve']=improve['Q1_Role']+"_"+improve['Q3_Improve']
prolst3 =improve['Age_Improve'].dropna().unique()
pro_nps3= nps_rolelevel(improve, prolst3,'Age_Improve')
pro_nps3= pro_nps3.sort(['NPS'], ascending=0)
#test =  pro_nps3[pro_nps3['NPS']<0.6]
#test.to_csv("test3.csv",index=False)
pro_nps3.to_csv("test5.csv",index=False)

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [33]:
price = improve[improve['Q3_Improve']=='Pricing'] ### Mostly Programmer
#Experienced programmer	167, New programmer	50

programmer = improve[improve['Q1_Role']=='Experienced programmer']
test = pd.DataFrame(programmer.groupby('Q3_Improve').size().reset_index())
test

,Q3_Improve,0
0,Code review,160
1,GitHub desktop apps,109
2,Help documentation,46
3,Other (please specify),170
4,Pricing,167
5,Project management tools,239
6,User administration tools,62


In [34]:
programmer['Age_Year2'] = programmer['github_age_in_days'].apply(lambda x : agebucket2(x))
programmer['Age_Improve']=programmer['Age_Year2']+"_"+programmer['Q3_Improve']
prolst3 = programmer['Age_Improve'].dropna().unique()
pro_nps3= nps_rolelevel(programmer, prolst3,'Age_Improve')
pro_nps3= pro_nps3.sort(['NPS'], ascending=0)
pro_nps3
test =  pro_nps3[pro_nps3['NPS']<0.5]
test

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



,Improvement,NPS,Total,Promoters,Promoter%,Passives,Passive%,Detractors,Detractor%
38,Y2_Help documentation,0.470588,17,11,0.647059,3,0.176471,3,0.176471
1,Y1_Help documentation,0.444444,9,5,0.555556,3,0.333333,1,0.111111
6,Y7_Code review,0.444444,9,5,0.555556,3,0.333333,1,0.111111
4,Y4_User administration tools,0.444444,9,6,0.666667,1,0.111111,2,0.222222
...,...,...,...,...,...,...,...,...,...
24,Y5_Pricing,0.318182,22,11,0.500000,7,0.318182,4,0.181818
41,Y6_Pricing,0.222222,9,3,0.333333,5,0.555556,1,0.111111
46,Y7_Pricing,0.166667,6,2,0.333333,3,0.500000,1,0.166667
5,Y1_Other (please specify),0.125000,24,8,0.333333,11,0.458333,5,0.208333


In [35]:
prolst2 = programmer['Q3_Improve'].dropna().unique()
pro_nps2= nps_rolelevel(programmer, prolst2,'Q3_Improve')
pro_nps2= pro_nps2.sort(['NPS'], ascending=0)
pro_nps2

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



,Improvement,NPS,Total,Promoters,Promoter%,Passives,Passive%,Detractors,Detractor%
0,Project management tools,0.732218,239,180,0.753138,54,0.225941,5,0.020921
4,User administration tools,0.645161,62,43,0.693548,16,0.258065,3,0.048387
6,Code review,0.643750,160,110,0.687500,43,0.268750,7,0.043750
2,GitHub desktop apps,0.614679,109,72,0.660550,32,0.293578,5,0.045872
1,Help documentation,0.608696,46,32,0.695652,10,0.217391,4,0.086957
5,Other (please specify),0.570588,170,112,0.658824,43,0.252941,15,0.088235
3,Pricing,0.514970,167,105,0.628743,43,0.257485,19,0.113772


In [36]:
prolst = programmer['Age_Year2'].dropna().unique()
pro_nps= nps_rolelevel(programmer, prolst,'Age_Year2')
pro_nps= pro_nps.sort(['Improvement'], ascending=1)
pro_nps

C:\Users\jiaqil\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



,Improvement,NPS,Total,Promoters,Promoter%,Passives,Passive%,Detractors,Detractor%
0,Y1,0.408333,120,65,0.541667,38,0.316667,16,0.133333
2,Y2,0.581633,196,128,0.653061,54,0.275510,14,0.071429
5,Y3,0.691304,230,169,0.734783,51,0.221739,10,0.043478
1,Y4,0.677725,211,152,0.720379,50,0.236967,9,0.042654
4,Y5,0.603448,116,77,0.663793,32,0.275862,7,0.060345
6,Y6,0.789474,57,46,0.807018,10,0.175439,1,0.017544
3,Y7,0.566667,30,19,0.633333,9,0.300000,2,0.066667


# Text Analysis

In [345]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [349]:
improve.columns.values

array(['github_age_in_days', 'Q1_Role', 'Q5_Score', 'Q6_Change',
       'Q3_Improve', 'Q11_Research', 'Q3_Improve_SelText', 'Q2_Impression',
       'Q4_Improve_Text', 'Age_Year2', 'Age_Improve'], dtype=object)

In [362]:
text= [i for i in improve['Q3_Improve_SelText'].dropna()]

In [363]:
text1= [i for i in improve['Q4_Improve_Text'].dropna()]
text2= [i for i in improve['Q6_Change'].dropna()]
text.append(text1)
text.append(text2)

In [368]:
text1 = [str(i).lower() for i in text]

In [370]:
stopset = set(stopwords.words('english'))

In [371]:
vectorizer = TfidfVectorizer(stop_words=stopset,
                                 use_idf=True, ngram_range=(1, 3))
X = vectorizer.fit_transform(text1)

In [372]:
lsa = TruncatedSVD(n_components=27, n_iter=100)
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=27, n_iter=100,
       random_state=None, tol=0.0)

In [375]:
import sys


In [377]:
terms = vectorizer.get_feature_names()
for i, comp in enumerate(lsa.components_): 
    termsInComp = zip (terms,comp)
    sortedTerms =  sorted(termsInComp, key=lambda x: x[1], reverse=True) [:10]
    print("Concept %d:" % i)
    for term in sortedTerms:
        print(term[0])
    print(" ")

Concept 0:
github
would
like
private
use
free
code
git
project
make
 
Concept 1:
yes
care
status
done
code review
thing
basically
review
pull requests
things
 
Concept 2:
free
private
free private
private repositories
repositories
free private repositories
repo
repos
private repos
better
 
Concept 3:
github
tools
way
good
make
apps linux
private
new
github apps
linux
 
Concept 4:
like
project
could
one
repositories
thanks
new
something
jira
windows
 
Concept 5:
github
pull
private
repos
requests
nice
linux
free private repositories
pull requests
see
 
Concept 6:
linux
make
app
great
better
pull
use
interface
pricing
tools
 
Concept 7:
tools
management
project
without
apps
app
git
repos
improve
work
 
Concept 8:
use
nice
git
repo
make
find
free private repositories
new
way
like
 
Concept 9:
use
good
features
repo
issues
private repos
windows
pull requests
projects
pull
 
Concept 10:
could
project
github
would
requests
way
app
page
without
awesome
 
Concept 11:
review
code
project
window

In [37]:
html = '''<html><head><link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.1/css/bootstrap.min.css"><style>body{ margin:0 100; background:whitesmoke; }</style><script src="https://cdn.plot.ly/plotly-latest.min.js"></script></head><body><h1>GitHub Survey Analysis</h1><h2>Section 1: General Survey Analysis</h2>''' + age_hist
html = html + " "+first+ " " + role_pie
html = html + '<br></br><br></br><h2>Section 2: NPS vs. Account Age Analysis</h2>'+nps_hist+" "+linear+" "+agenps
html = html + '<br></br><br></br><h2>Section 3: Customer Role & NPS Analysis</h2>'+rolenps+" "+meanagenps
html = html + '<br></br><br></br><h2>Section 4: Improvement & NPS Analysis</h2>' + imp_pie+ " " +impnps +" "+impmeannps+'</body></html>'

In [38]:
file= open("test.html","w")
file.write(html)
file.close()